In [1]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import plotly.offline as pyo
import numpy as np
import numpy as np
import os

from tensorflow.keras.models import load_model

pd.set_option("display.max_columns", None)

def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(0, 250, 0, .9)'
        ),
        name = 'target'
    )
    fig.add_trace(tg, row=1, col=1)
    pyo.plot(fig, filename="prova.html", auto_open=True)



In [2]:
learning_rate = 0.0001
batch_size = "max"
look_back = 60
file_modello = "LSTM_singola.keras"

In [3]:
elenco_usin = pd.read_parquet("Tickers_De_Giro.parquet")
elenco_usin = elenco_usin.loc[elenco_usin["Ticker"] == "BTG", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
tot_usin = len(elenco_usin)
i = 1
if os.path.exists(file_modello):
    model = load_model(file_modello)
    print("Modello esistente")
else:
    model = 0
    print("Nuovo modello")
for index, riga in elenco_usin.iterrows():       
    print(f"\r{i} di {tot_usin}) Caricamento ticker {riga['Ticker']}                           \n", end=' ', flush=True)
    ticker = yf.download(riga['USIN'], start='2005-01-01', end='2019-07-31', progress=False)
    if len(ticker) < 500:
        continue
    else:
        ticker = fx.crea_indicatori(ticker)
        features, target = fx.features_e_target(ticker)
        ticker["Target"] = target
        print(f"\r{i} di {tot_usin}) Addestramento ticker {riga['Ticker']}                           \n", end=' ', flush=True)
        try:
            model = fx.addestramento(model, features=features, target=target, learning_rate=learning_rate, batch_size=batch_size, look_back=look_back) 
            model.save(file_modello)
        except Exception as e:
            print(e)
            continue
        i += 1

print("fine addestramento")

Modello esistente
1 di 1) Caricamento ticker BTG                           
1 di 1) Addestramento ticker BTG                           
 [ 0.52048528 12.7038835 ]
Epoch 1/100
1/1 [==============================] - 17s 17s/step - loss: 5.5862 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4871 - accuracy: 0.9470 - val_loss: 3.1759 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6731 - val_accuracy: 0.9389
Epoch 2/100
1/1 [==============================] - 7s 7s/step - loss: 5.3950 - precision: 0.0571 - recall: 0.0506 - auc: 0.5295 - accuracy: 0.9326 - val_loss: 3.1774 - val_precision: 0.1429 - val_recall: 0.1667 - val_auc: 0.6950 - val_accuracy: 0.9160
Epoch 3/100
1/1 [==============================] - 8s 8s/step - loss: 4.9623 - precision: 0.0896 - recall: 0.1519 - auc: 0.6395 - accuracy: 0.9097 - val_loss: 3.2217 - val_precision: 0.2174 - val_recall: 0.4167 - val_auc: 0.7665 - val_accuracy: 0.9046
Epoch 4/100
1/1 [==============================] - 8s 8s/step -